### TRAINING MODELS

In [2]:
# IMPORT EVERYTHING
from Architectures.AE_ADJ_v2 import AE

from Data.load import load, load_middle_slices
from Visualizations.latent_space_projections import plot_multiple_datasets
from Visualizations.plots import plot_middle_slices_in_range, plot_models_training_time, compare_models_loss_history, plot_images, compare_models_reconstruction
from Metrics.metrics_tf import MSE_loss, NMSE_loss, NRMSE_loss, SSIM_loss
from Metrics.metrics import NMSE, SSIM, NRMSE, MSE

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, TSNE
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import layers, Model, activations, regularizers
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from skimage.metrics import structural_similarity as ssim

import umap
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pandas as pd
import seaborn as sns
import glob
import tensorflow as tf
import time
import math
import nibabel as nib
import json
from tqdm import tqdm

c:\Users\kiran\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#groups = {}
groups3D = {}
for group in tqdm(["CN","MCI","AD"],"Opening faster"):
    #groups[group]=np.load(open(f"Data/D2-{group}.npy","rb"))
    groups3D[group]=np.load(open(f"Data/D3-{group}.npy","rb"))

Opening faster: 100%|██████████| 3/3 [00:09<00:00,  3.05s/it]


In [ ]:
"""
A2 = AE(name="A2", input_shape=groups["CN"].shape, latent_dim=16,
    encoder_layers=[
        (layers.Conv2D, 16, (3,3), {'strides': 2, 'padding': 'same', 'activation': 'leaky_relu'}),
        (layers.Conv2D, 32, (3,3), {'strides': 2, 'padding': 'same', 'activation': 'leaky_relu'}),
    ],
    decoder_layers=[
        (layers.Conv2D, 32, (3,3), {'padding': 'same'}), (layers.UpSampling2D, (2,2)),
        (layers.Conv2D, 16, (3,3), {'padding': 'same'}), (layers.UpSampling2D, (2,2)),
        (layers.Conv2D, 1, (3,3), {'padding': 'same'}),
    ], VAE_model=False, optimizer="adam", loss_fn="mse").compile_model()
"""

"""
public_repo_mse_2d = AE(name="public_repo_2D_mse", input_shape=groups["CN"].shape, latent_dim=128,
    encoder_layers=[
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (2, 2)}),
        (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (2, 2)}),
        (layers.Conv2D, 128, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 128, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization()
    ],
    decoder_layers=[
        (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2D, 1,  (3, 3), {'activation': 'sigmoid','padding': 'same',}),
    ],
    VAE_model=False, optimizer="adam", loss_fn=MSE_loss).compile_model()

public_repo_ssim_2d = AE(name="public_repo_2D_ssim", input_shape=groups["CN"].shape, latent_dim=128,
    encoder_layers=[
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (2, 2)}),
        (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (2, 2)}),
        (layers.Conv2D, 128, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 128, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization()
    ],
    decoder_layers=[
        (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
        (layers.Conv2D, 1,  (3, 3), {'activation': 'sigmoid','padding': 'same',}),
    ],
    VAE_model=False, optimizer="adam", loss_fn=SSIM_loss).compile_model()
"""

public_repo_3d = AE(name="public_repo_autoencoder_3D_mse", input_shape=groups3D["CN"].shape, latent_dim=16,
    encoder_layers=[
        (layers.Conv3D, 32, (3, 3,3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv3D, 32, (3, 3,3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling3D, {'pool_size': (2, 2, 2)}),
        (layers.Conv3D, 64, (3, 3,3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv3D, 64, (3, 3,3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling3D, {'pool_size': (2, 2, 2)}),
        (layers.Conv3D, 128, (3, 3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv3D, 128, (3, 3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization()
    ],
    decoder_layers=[
        (layers.Conv3D, 64, (3, 3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv3D, 64, (3, 3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling3D, {'size': (2, 2, 2)}),
        (layers.Conv3D, 32, (3, 3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
        (layers.Conv3D, 32, (3, 3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling3D, {'size': (2, 2, 2)}),
        (layers.Conv3D, 1,  (3, 3, 3), {'activation': 'sigmoid','padding': 'same',}),
    ],
    VAE_model=False, optimizer="adam", loss_fn=MSE_loss).compile_model()


Interpreted image shape: (80, 96, 80, 1) Pre-flattened latent shape: (None, 20, 24, 20, 128)


In [ ]:
for m in [public_repo_3d]: #[A2,A4,A5]
    m.train(groups3D["CN"], epochs=2, batch_size=16, verbose=True, 
    save_path=r"C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\AE_ADJ_models\newer_images", 
    save_interval=1000, patience=30)

Training public_repo_autoencoder_3D_mse on CPU
Input shape: (988, 80, 96, 80)
 7/50 ━━━━━━━━━━━━━━━━━━━━ 41:06 57s/step - loss: 0.1490

In [ ]:
# Test to see if loading works
# models_list = [AE.open(r"C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\AE_ADJ_models\newer_images\public_repo\epoch_9")]

Models loaded successfully.
